In [1]:
import KeyFunctions as kf

root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

In [28]:
#Import the postcode list from the raw data gdf
gdf = kf.load_obj(root_path,"raw_gdf_StPauls")

In [29]:
#gdf["Type"].unique().tolist()

In [31]:
#Initialise our postcodes gdf as a list of postcodes and their points
Postcodes = gdf.loc[gdf["Type"]=="Postcodes",["Name", "geometry"]].copy()
Postcodes = Postcodes.rename(columns={"Name": "Postcode"})

In [32]:
#List out the points we will want to use for densities, point distances...etc.

All_Crimes = gdf.loc[gdf["Type"]=="Crimes",:]
All_Schools = gdf.loc[gdf["Type"]=="Schools",:]
Hospitals = gdf.loc[gdf["Type"]=="Hospital",:]
RailwayStations = gdf.loc[gdf["Type"]=="RailwayStations",:]
#...etc.

#Point datasets to find the density of and the radius of those densities in the format output_name:dataframe where the output variable will have the name Outname_Density_Radius
density_var_dict = {"Postcodes":Postcodes, "All_Crimes":All_Crimes, "All_Schools":All_Schools, "Hospitals":Hospitals, "RailwayStations":RailwayStations}
density_r_list = [1000,5000,25000]

#Point datasets to find the nearest of in the format output_name:dataframe where the output variable will have the name Nearest_Out_Name
nearest_var_dict = {"School":All_Schools, "Hospital":Hospitals, "RailwayStation":RailwayStations}

In [33]:
#Start with adding density columns
for k in density_var_dict:
    for r in density_r_list:
        Postcodes[k + "_Density_" + str(r)] = kf.density_within_radius(Postcodes,density_var_dict[k], r)

In [37]:
#Add distance to the nearest, don't include geometry lines
keep_n_g_d = [False, False, True]
for k in nearest_var_dict:
    Postcodes = kf.nearest_neighbor_point(Postcodes, nearest_var_dict[k], "Name", k, keep_n_g_d = keep_n_g_d, merge=True)

In [38]:
Postcodes.columns

Index(['Postcode', 'geometry', 'Postcodes_Density_1000',
       'Postcodes_Density_5000', 'Postcodes_Density_25000',
       'All_Crimes_Density_1000', 'All_Crimes_Density_5000',
       'All_Crimes_Density_25000', 'All_Schools_Density_1000',
       'All_Schools_Density_5000', 'All_Schools_Density_25000',
       'Hospitals_Density_1000', 'Hospitals_Density_5000',
       'Hospitals_Density_25000', 'RailwayStations_Density_1000',
       'RailwayStations_Density_5000', 'RailwayStations_Density_25000',
       'Nearest_Crime_Distance', 'Nearest_School_Distance',
       'Nearest_Hospital_Distance', 'Nearest_RailwayStation_Distance'],
      dtype='object')

In [39]:
Postcodes.head()

,Postcode,geometry,Postcodes_Density_1000,Postcodes_Density_5000,Postcodes_Density_25000,All_Crimes_Density_1000,All_Crimes_Density_5000,All_Crimes_Density_25000,All_Schools_Density_1000,All_Schools_Density_5000,...,Hospitals_Density_1000,Hospitals_Density_5000,Hospitals_Density_25000,RailwayStations_Density_1000,RailwayStations_Density_5000,RailwayStations_Density_25000,Nearest_Crime_Distance,Nearest_School_Distance,Nearest_Hospital_Distance,Nearest_RailwayStation_Distance
950663,CM233WE,POINT (530754.000 182754.000),6038,33949,34071,21848,166699,168016,21,158,...,3,13,13,4,20,20,57.717418,245.065297,746.207121,516.364213
1200049,E11AB,POINT (534253.000 181031.000),2171,33935,34071,14014,167159,168016,18,155,...,0,12,13,1,20,20,29.426898,185.148589,1137.880247,847.897399
1200051,E11AE,POINT (534279.000 181467.000),1968,33704,34071,16946,165690,168016,19,156,...,0,12,13,0,20,20,27.669988,128.763349,1209.391430,1016.311960
1200052,E11AF,POINT (534253.000 181031.000),2171,33935,34071,14014,167159,168016,18,155,...,0,12,13,1,20,20,29.426898,185.148589,1137.880247,847.897399
1200053,E11AG,POINT (534291.000 181451.000),1915,33694,34071,16631,165690,168016,19,156,...,0,12,13,0,20,20,47.391731,119.603512,1216.519838,1018.470422
